In [ ]:
import sys
import time

sys.path.append("..")  # make sure we can run this from the repo
from IPython.display import Markdown, clear_output, display
from ipywidgets import widgets

import cohere
CHAT_MODEL = 'converse-xlarge-nightly'

## Client example

### Simple Generate Example

In [ ]:
co = cohere.Client()
start_time = time.time()
streaming_gens = co.generate(prompt="Hey! Don't worry, 🐝 happy~", max_tokens=20, stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

### Widgets Chat Example

In [ ]:
co = cohere.Client()
max_tokens = 250

text = ""
output_area = display("Type your message below and hit enter to chat!", display_id=True)
input_text = widgets.Text(value="Hey!", description="User:", disabled=False)

def append_text(s):
    global text
    text += s
    output_area.update(Markdown(text))

def stream_response(input_text):
    query = input_text.value
    if query.startswith("<generating"):
        return
    input_text.value = "<generating response>"
    append_text(f"\n\nUser: {query}\n\nChatbot: ")
    streaming_chat = co.chat(
        query=query,
        max_tokens=max_tokens,
        model=CHAT_MODEL,
        stream=True,
    )
    for token in streaming_chat:
        append_text(token.text)
    input_text.value = ""

display(input_text)
input_text.on_submit(stream_response)

## AsyncClient example

### Widgets generate example

In [ ]:
prompt = "안녕하"
num_generations = 5
max_tokens = 50
texts = [f"**Generation #{i+1}:** {prompt}" for i in range(num_generations)]
displays = [display(display_id=True) for t in texts]

async with cohere.AsyncClient() as aio_co:
    start_time = time.time()
    aio_streaming_gens = await aio_co.generate(
        prompt=prompt,
        max_tokens=max_tokens,
        model="command-xlarge-nightly",
        num_generations=num_generations,
        stream=True,
    )

    async for token in aio_streaming_gens:
        texts[token.index] += token.text
        displays[token.index].update(Markdown(texts[token.index]))

### Simple Chat Example

In [ ]:
query = "Do You Want to Build a Snowman?"
async with cohere.AsyncClient() as aio_co:
    streaming_chat = await aio_co.chat(
        query=query,
        max_tokens=50,
        model=CHAT_MODEL,
        stream=True,
    )
    async for token in streaming_chat:
        print(token)
    print(streaming_chat.texts)